In [1]:
# Load packages
import os
import pandas as pd
import json
import spacy
from spacy.matcher import Matcher
nlp=spacy.load("en_core_web_sm")

In [ ]:
# Load and Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

os.chdir(" ") #change directory
os.listdir() # Check

Mounted at /content/drive


['test.xlsx',
 'mwe.json',
 'Test2.xlsx',
 'colab_pdf.py',
 '__pycache__',
 'tfidf_movies.csv',
 'with_similarity.csv',
 'old_data',
 'Test3.xlsx',
 'mwe2.json',
 'MovieDataClean2.xlsx',
 'intensifiers.xlsx',
 'mwedict.json',
 'MovieData_Spacy.xlsx']

In [3]:
# Load new data:

df2 = pd.read_excel("MovieDataClean2.xlsx", index_col=0)
df2.head()

,MovieID,MovieTitle,MovieYear,NWriters,WriterGender,Gender,Text
Column1,,,,,,,
1,m0,10 things i hate about you,1999,2,W,F,They do not! I hope so. Let's go. Okay -- you'...
2,m0,10 things i hate about you,1999,2,W,M,"They do to! She okay? Wow No The ""real you"". I..."
3,m1,1492: conquest of paradise,1992,1,W,F,NaN
4,m1,1492: conquest of paradise,1992,1,W,M,Asia can be found to the west -- and I will pr...
5,m10,affliction,1997,1,M,F,All the figures show is that Gordon LaRiviere ...


In [4]:
# Import list of intensifiers
dfintensifiers = pd.read_excel("intensifiers.xlsx", sheet_name="intensifiers_singleword")
print(dfintensifiers.head())
intensifierslist = list(dfintensifiers.Intensifier)

  Intensifier        Source
0  absolutely  Fuchs (2017)
1         all  Fuchs (2017)
2  altogether  Fuchs (2017)
3   amazingly  Fuchs (2017)
4       awful  Fuchs (2017)


In [5]:
# Dictionary of multiword expressions - original
with open('mwe.json', 'r') as file:
  mwe_dict = json.load(file)

In [6]:
# Safe divide

def safe_divide(a, b):
    if b != 0: #
        return a/b
    else:
        return 0

In [7]:
# Use on test data
dftest = pd.read_excel("Test3.xlsx", index_col=0)
dftest.head()

,Text,MovieTitle,Gender,MovieYear
MovieID,,,,
1,This does not contain any intensifiers or mult...,"Hello, sup",m,1999
1,I am really very extremely supremely happy today.,"Hello, sup",f,1999
2,"By and large, this sentence contains an awful ...","Hello, sup",m,1999
2,This is a damned sight better than I thought.,"Hello, sup",f,1999


In [8]:
# NLP pipeline on test data
test_docs = list(nlp.pipe(dftest.Text))

In [9]:
## Test method 1 (original)

nw_final = [] # word count
intensifiers_final = [] # Modal count

matcher = Matcher(nlp.vocab)
for entry in mwe_dict:
    matcher.add(entry["name"], [entry["pattern"]])

#loop for each document in spacy doc
for doc in test_docs:
    nw = 0 #starting count for number of words
    intensifiers = 0

    for token in doc: #for token in each doc
      #  print(token.text, token.pos_)
        if token.is_punct or token.is_space:
          continue # skip punctuation and spaces
        nw += 1

    # Count MWE matches and mark covered token indices
    matches = matcher(doc)
    intensifiers += len(matches)
    print("Length:", intensifiers)

    covered = set()
    for _, start, end in matches:
        covered.update(range(start, end)) # tokens that are part of mwe matches

    # Count single-word intensifiers, skipping tokens inside MWEs
    for i, token in enumerate(doc):
        print(token.text, token.pos_)  # sanity check
        if token.is_punct or token.is_space:
            continue
        if i in covered:
            continue
        if token.pos_ == "ADV" and token.lower_ in intensifierslist:
            print("this is an intensifier")
            intensifiers += 1

    nw_final.append(nw) #append number of words by text to list above (creating a list of intensifiers)
    intensifiers_final.append(safe_divide(intensifiers, nw))

dftest['NW'] = nw_final
dftest['intensifiers'] = intensifiers_final

Length: 0
This PRON
does AUX
not PART
contain VERB
any DET
intensifiers NOUN
or CCONJ
multiword NOUN
expressions NOUN
. PUNCT
Length: 0
I PRON
am AUX
really ADV
this is an intensifier
very ADV
this is an intensifier
extremely ADV
this is an intensifier
supremely ADV
this is an intensifier
happy ADJ
today NOUN
. PUNCT
Length: 1
By ADP
and CCONJ
large ADJ
, PUNCT
this DET
sentence NOUN
contains VERB
an DET
awful ADJ
lot NOUN
of ADP
intensifiers NOUN
. PUNCT
Including VERB
20 NUM
times NOUN
as ADV
many ADJ
as ADP
before ADV
. PUNCT
Length: 1
This PRON
is AUX
a DET
damned ADJ
sight NOUN
better ADV
than SCONJ
I PRON
thought VERB
. PUNCT


In [10]:
dftest

,Text,MovieTitle,Gender,MovieYear,NW,intensifiers
MovieID,,,,,,
1,This does not contain any intensifiers or mult...,"Hello, sup",m,1999,9,0.000000
1,I am really very extremely supremely happy today.,"Hello, sup",f,1999,8,0.500000
2,"By and large, this sentence contains an awful ...","Hello, sup",m,1999,18,0.055556
2,This is a damned sight better than I thought.,"Hello, sup",f,1999,9,0.111111


Even though the sentence beginning By and large... has 3 intensifier phrases, it only counts one of them using the original part of speech tags.

An alternate approach using dependency parsing might be more effective:

In [11]:
# check dependencies before trying a dependency parsing based method
for doc in test_docs:
  for token in doc:
    print (token, token.dep_)

This nsubj
does aux
not neg
contain ROOT
any det
intensifiers dobj
or cc
multiword compound
expressions conj
. punct
I nsubj
am ROOT
really advmod
very advmod
extremely advmod
supremely advmod
happy acomp
today npadvmod
. punct
By prep
and cc
large conj
, punct
this det
sentence nsubj
contains ROOT
an det
awful amod
lot dobj
of prep
intensifiers pobj
. punct
Including ROOT
20 pobj
times quantmod
as advmod
many amod
as prep
before pcomp
. punct
This nsubj
is ROOT
a det
damned amod
sight attr
better advmod
than mark
I nsubj
thought advcl
. punct


In [12]:
from spacy import displacy

for doc in test_docs:
  displacy.render(doc, jupyter = True, style='dep')

In [13]:
# Check how these expressions are being parsed - each sentence has a multiword expression from the mwe list used in the project.

intensifier_sentences = [
    ["This is by far the best dessert I've ever had."],
    ["That was quite a performance."],
    ["I've never seen quite an expression."],
    ["It was 10 times bigger than I imagined!"],
    ["That was a darn sight harder than expected."],
    ["She's a great deal happier now."],
    ["That was a hell of a throw."],
    ["This place has a hell of a lot more to offer."],
    ["I like her a lot."],
    ["He's got a whole lot of explaining to do."],
    ["He proved himself above all doubt."],
    ["This was all together magnificent."],
    ["I care an awful lot about this cause."],
    ["The cat is as curious as anything can be."],
    ["That was funny as hell!"],
    ["The project is by and large finished."],
    ["It's a darn sight better than before."],
    ["You're a darned sight smarter than me."],
    ["The painting is ever so colorful."],
    ["He is every bit as much an artist as I am."],
    ["The coast is far away from here."],
    ["For the life of me, I can't remember."],
    ["For the most part, I'm satisfied."],
    ["She scared the hell out of the children."],
    ["In the main, this strategy works."],
    ["You were like hell trying to get that done!"],
    ["She lives a long way away from here."],
    ["He grew more and more weary by the day."],
    ["He demands more and more and more."],
    ["This movie captivated me more so than the last one."],
    ["She likes that most of all."],
    ["This isn't half as difficult as it seemed."],
    ["On the whole, life is good."],
    ["He did that something close to perfect."],
    ["The cold weather is something terrible today."],
    ["Such a beautiful landscape!"],
    ["He succeeded to a large extent."],
    ["He isn't interested to any great extent in sports."],
    ["She loves him to bits."],
    ["The dog was scared to death by the fireworks."],
    ["He loved her to pieces."],
    ["This is ridiculous to the nth degree."],
    ["Their courage is to their nth degree unmatched."],
    ["That was a heck of a presentation!"],
    ["The odyssey was a damned sight more challenging."],
    ["I've only seen a damn sight fewer stars tonight."],
    ["I appreciated the damn sight wisdom in his words."],
    ["There's a good deal of truth in that story."],
    ["I've got a fair bit of work to do."],
    ["You missed out on a good bit of fun."],
    ["This task is not challenging at all."],
    ["We're pretty much ready to go."],
    ["The truth hit him stone cold in the face."],
    ["She climbed right up the wall energetically."],
    ["He is an out and out genius!"],
    ["He's a great person through and through."],
    ["The wind blew the snow clean off the roof."],
    ["That was flat out the craziest thing I've seen!"],
    ["She left right away after the meeting."],
    ["His actions were right proper, indeed."],
    ["There was a fair deal of screaming involved."]
]

In [14]:
flattened_sentences = [sentence[0] for sentence in intensifier_sentences]

# Process the list of sentences with spaCy
nlp_docs = [nlp(sentence) for sentence in flattened_sentences]

In [15]:
# for doc in nlp_docs:
#   displacy.render(doc, jupyter = True, style='dep')
#   for token in doc:
#     print(token, token.dep_)

In [16]:
# Now perform versions of the pipeline on this list:
# (1) Original pipeline with POS tagging
# (2) Alternate pipeline with dependency parsing

dfmwe = pd.read_excel("intensifiers.xlsx", sheet_name="intensifiers_mwe")
print(dfmwe.head())
mwelist = list(dfmwe.Intensifier)

    Intensifier        Source
0        by far  Fuchs (2017)
1       quite a  Fuchs (2017)
2      quite an  Fuchs (2017)
3   *Num* times          USAS
4  a darn sight          USAS


In [17]:
dfsents = pd.DataFrame(flattened_sentences, columns=['text'])

In [18]:
C## Test method 1 again on these sentences

nw_final = [] # word count
intensifiers_final = [] # Modal count

matcher = Matcher(nlp.vocab)
for entry in mwe_dict:
    matcher.add(entry["name"], [entry["pattern"]])

#loop for each document in spacy doc
for doc in nlp_docs:
    nw = 0 #starting count for number of words
    intensifiers = 0

    for token in doc: #for token in each doc
      #  print(token.text, token.pos_)
        if token.is_punct or token.is_space:
          continue # skip punctuation and spaces
        nw += 1

    # Count MWE matches and mark covered token indices
    matches = matcher(doc)
    intensifiers += len(matches)
    #print("Length:", intensifiers)

    covered = set()
    for _, start, end in matches:
        covered.update(range(start, end)) # tokens that are part of mwe matches

    # Count single-word intensifiers, skipping tokens inside MWEs
    for i, token in enumerate(doc):
        #print(token.text, token.pos_)  # sanity check
        if token.is_punct or token.is_space:
            continue
        if i in covered:
            continue
        if token.pos_ == "ADV" or token.pos_ == "ADJ" and token.lower_ in intensifierslist:
            #print("this is an intensifier")
            intensifiers += 1

    nw_final.append(nw) #append number of words by text to list above (creating a list of intensifiers)
    intensifiers_final.append(safe_divide(intensifiers, nw))

dfsents['NW'] = nw_final
dfsents['intensifiers'] = intensifiers_final

In [19]:
dfsents

,text,NW,intensifiers
0,This is by far the best dessert I've ever had.,11,0.272727
1,That was quite a performance.,5,0.000000
2,I've never seen quite an expression.,7,0.142857
3,It was 10 times bigger than I imagined!,8,0.000000
4,That was a darn sight harder than expected.,8,0.250000
...,...,...,...
56,The wind blew the snow clean off the roof.,9,0.000000
57,That was flat out the craziest thing I've seen!,10,0.000000
58,She left right away after the meeting.,7,0.285714
59,"His actions were right proper, indeed.",6,0.333333


In [20]:
# Method 2 - use dependency parsing

from spacy.matcher import PhraseMatcher

# Build a PhraseMatcher for exact phrase matches (case-insensitive)
phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(phrase) for phrase in mwelist]  # mwelist already lowercase strings
phrase_matcher.add("MWE", patterns)

nw_final = []  # word count
intensifiers_final = []  # per million words

relevant_deps = {"advmod", "amod", "npadvmod", "advcl"}

for doc in nlp_docs:
    # Count non-space, non-punct tokens
    nw = sum(1 for t in doc if not (t.is_punct or t.is_space))
    intensifiers = 0

    # Exact phrase matches
    matches = phrase_matcher(doc)  # list of (match_id, start, end)

    # Deduplicate overlaps longest-first and filter by dependency labels
    # Convert to spans for convenience
    spans = [(start, end) for _, start, end in matches]
    # Sort by span length (end - start) descending
    spans.sort(key=lambda se: (se[1] - se[0]), reverse=True)

    covered = set()
    selected_spans = []
    for start, end in spans:
        # Skip if any token overlaps with already selected spans
        if any(i in covered for i in range(start, end)):
            continue
        # Check dependency labels inside the span
        if any(doc[i].dep_ in relevant_deps for i in range(start, end)):
            selected_spans.append((start, end))
            covered.update(range(start, end))

    # Count MWEs
    intensifiers += len(selected_spans)

    # Count single-word intensifiers, skipping tokens inside MWEs
    for i, token in enumerate(doc):
        if token.is_punct or token.is_space:
            continue
        if i in covered:
            continue
        if token.pos_ == "ADV" or token.pos_=="ADJ" and token.lower_ in intensifierslist:
            intensifiers += 1

    nw_final.append(nw)
    intensifiers_final.append(safe_divide(intensifiers, nw))

dfsents["NW2"] = nw_final
dfsents["intensifiers2"] = intensifiers_final

In [21]:
dfsents

,text,NW,intensifiers,NW2,intensifiers2
0,This is by far the best dessert I've ever had.,11,0.272727,11,0.272727
1,That was quite a performance.,5,0.000000,5,0.000000
2,I've never seen quite an expression.,7,0.142857,7,0.142857
3,It was 10 times bigger than I imagined!,8,0.000000,8,0.000000
4,That was a darn sight harder than expected.,8,0.250000,8,0.250000
...,...,...,...,...,...
56,The wind blew the snow clean off the roof.,9,0.000000,9,0.111111
57,That was flat out the craziest thing I've seen!,10,0.000000,10,0.000000
58,She left right away after the meeting.,7,0.285714,7,0.142857
59,"His actions were right proper, indeed.",6,0.333333,6,0.333333


Comparing the two methods showed that using POS tagging was more effective than
using dependency parsing - however, not all expressions were being detected
effectively.

In [22]:
# Compare - original method with pos tagging was more effective -
# but still did not capture everything

count_method1 = dfsents[dfsents['intensifiers'] > 0].shape[0]
count_method2 = dfsents[dfsents['intensifiers2'] > 0].shape[0]

print(f"Method 1 detected: {count_method1}")
print(f"Method 2 detected: {count_method2}")

Method 1 detected: 48
Method 2 detected: 43


In [23]:
# check sentences that were not captured using original method
display(dfsents[dfsents['intensifiers'] == 0])

,text,NW,intensifiers,NW2,intensifiers2
1,That was quite a performance.,5,0.0,5,0.000000
3,It was 10 times bigger than I imagined!,8,0.0,8,0.000000
9,He's got a whole lot of explaining to do.,10,0.0,10,0.100000
14,That was funny as hell!,5,0.0,5,0.000000
15,The project is by and large finished.,7,0.0,7,0.142857
24,"In the main, this strategy works.",6,0.0,6,0.000000
33,He did that something close to perfect.,7,0.0,7,0.142857
36,He succeeded to a large extent.,6,0.0,6,0.166667
41,This is ridiculous to the nth degree.,7,0.0,7,0.000000
42,Their courage is to their nth degree unmatched.,8,0.0,8,0.000000


In [24]:
# check their POS tags and compare to POS tags in the dictionary
selected_indices = [1, 2, 3, 9, 10, 14, 15, 22, 24, 30, 33, 36, 41, 42, 54, 56,
                    57, 60]
# This list is longer than the cell above because adjectives were not originally
# part of the pipeline for single-word identifiers.

for i in selected_indices:
    print(f"\n--- Document {i} ---")
    doc = nlp_docs[i]
    for token in doc:
      print(token.text, token.pos_)


--- Document 1 ---
That PRON
was AUX
quite DET
a DET
performance NOUN
. PUNCT

--- Document 2 ---
I PRON
've AUX
never ADV
seen VERB
quite DET
an DET
expression NOUN
. PUNCT

--- Document 3 ---
It PRON
was AUX
10 NUM
times NOUN
bigger ADJ
than SCONJ
I PRON
imagined VERB
! PUNCT

--- Document 9 ---
He PRON
's AUX
got VERB
a DET
whole ADJ
lot NOUN
of ADP
explaining VERB
to PART
do VERB
. PUNCT

--- Document 10 ---
He PRON
proved VERB
himself PRON
above ADP
all PRON
doubt ADV
. PUNCT

--- Document 14 ---
That PRON
was AUX
funny ADJ
as ADP
hell NOUN
! PUNCT

--- Document 15 ---
The DET
project NOUN
is AUX
by ADP
and CCONJ
large ADJ
finished NOUN
. PUNCT

--- Document 22 ---
For ADP
the DET
most ADJ
part NOUN
, PUNCT
I PRON
'm AUX
satisfied ADJ
. PUNCT

--- Document 24 ---
In ADP
the DET
main ADJ
, PUNCT
this DET
strategy NOUN
works VERB
. PUNCT

--- Document 30 ---
She PRON
likes VERB
that SCONJ
most ADJ
of ADP
all PRON
. PUNCT

--- Document 33 ---
He PRON
did VERB
that PRON
something PRO

In [25]:
# cell to test out small changes to see if it's an artifact of strange parsing of those particular sentences
sent = ["she's gone clean off her rocker", "I enjoyed that a fair deal",
        "Apples are 10 times better than oranges",
        "He pushed them to their nth degree"]
five_doc = list(nlp.pipe(sent))
for doc in five_doc:
  for token in doc:
    print(token.text, token.pos_)

she PRON
's AUX
gone VERB
clean ADJ
off ADP
her PRON
rocker NOUN
I PRON
enjoyed VERB
that SCONJ
a DET
fair ADJ
deal NOUN
Apples NOUN
are AUX
10 NUM
times NOUN
better ADJ
than ADP
oranges NOUN
He PRON
pushed VERB
them PRON
to ADP
their PRON
nth NOUN
degree NOUN


In [27]:
# Now try with updated list

# Dictionary of multiword expressions
with open('mwedict.json', 'r') as file:
  mwe_dict3 = json.load(file)

In [28]:
nw_final = [] # word count
intensifiers_final = [] # Modal count

matcher = Matcher(nlp.vocab)
for entry in mwe_dict3:
    matcher.add(entry["name"], [entry["pattern"]])

#loop for each document in spacy doc
for doc in nlp_docs:
    nw = 0 #starting count for number of words
    intensifiers = 0

    for token in doc: #for token in each doc
      #  print(token.text, token.pos_)
        if token.is_punct or token.is_space:
          continue # skip punctuation and spaces
        nw += 1

    # Count MWE matches and mark covered token indices
    matches = matcher(doc)
    intensifiers += len(matches)
    #print("Length:", intensifiers)

    covered = set()
    for _, start, end in matches:
        covered.update(range(start, end)) # tokens that are part of mwe matches

    # Count single-word intensifiers, skipping tokens inside MWEs
    for i, token in enumerate(doc):
        #print(token.text, token.pos_)  # sanity check
        if token.is_punct or token.is_space:
            continue
        if i in covered:
            continue
        if token.pos_ == "ADV" and token.lower_ in intensifierslist:
            #print("this is an intensifier")
            intensifiers += 1

    nw_final.append(nw) #append number of words by text to list above (creating a list of intensifiers)
    intensifiers_final.append(safe_divide(intensifiers, nw))

dfsents['NW3'] = nw_final
dfsents['intensifiers3'] = intensifiers_final

In [29]:
dfsents

,text,NW,intensifiers,NW2,intensifiers2,NW3,intensifiers3
0,This is by far the best dessert I've ever had.,11,0.272727,11,0.272727,11,0.090909
1,That was quite a performance.,5,0.000000,5,0.000000,5,0.200000
2,I've never seen quite an expression.,7,0.142857,7,0.142857,7,0.142857
3,It was 10 times bigger than I imagined!,8,0.000000,8,0.000000,8,0.125000
4,That was a darn sight harder than expected.,8,0.250000,8,0.250000,8,0.250000
...,...,...,...,...,...,...,...
56,The wind blew the snow clean off the roof.,9,0.000000,9,0.111111,9,0.000000
57,That was flat out the craziest thing I've seen!,10,0.000000,10,0.000000,10,0.100000
58,She left right away after the meeting.,7,0.285714,7,0.142857,7,0.142857
59,"His actions were right proper, indeed.",6,0.333333,6,0.333333,6,0.333333


In [30]:
count_method1 = dfsents[dfsents['intensifiers'] > 0].shape[0]
count_method2 = dfsents[dfsents['intensifiers2'] > 0].shape[0]
count_method3 = dfsents[dfsents['intensifiers3'] > 0].shape[0]

print(f"Method 1 detected: {count_method1}")
print(f"Method 2 detected: {count_method2}")
print(f"Method 3 detected: {count_method3}")

# Everything except the "clean off" sentence is now detected. Will leave that one as it is,
# because it's a weird case of clean being detected as a verb. Works if sentence is changed.

Method 1 detected: 48
Method 2 detected: 43
Method 3 detected: 60
